In [ ]:
from scipy.sparse.linalg import LinearOperator
import numpy as np

from matplotlib.pylab import plt

from dataclasses import dataclass

from luescher_nd.utilities import Solver

from luescher_nd.utilities import get_laplace_coefficients

In [ ]:
def myplus(a):
    return a + 1

In [ ]:
n = 10
lop = LinearOperator((n,n), matvec=myplus)

In [ ]:
a = np.arange(0, n)

In [ ]:
lop(a)

In [ ]:
%load_ext blackcellmagic

In [ ]:
@dataclass
class FFT_H:
    n1d: int
    epsilon: float
    ndim: int
    c0: float
    m: int
    cuda: bool = False

    def __post_init__(self):
        self.p1d = (
            np.append(np.arange(self.n1d // 2 + 1), np.arange(-self.n1d // 2 + 1, 0))
            * 2
            * np.pi
            / self.n1d
            / self.epsilon
        )
        self.p2 = np.sum(np.array(np.meshgrid(*[self.p1d] * self.ndim)) ** 2, axis=0)
        self.p2_over_2m = self.p2 / 2 / self.m

        self.mat = LinearOperator(
            matvec=self.apply_H, shape=[self.n1d ** self.ndim] * 2
        )

        self.p2_over_2m_device = cp.array(self.p2_over_2m) if self.cuda else None

        self.matp = LinearOperator(
            matvec=self.apply_Hp, shape=[self.n1d ** self.ndim] * 2
        )

    def apply_H(self, vec):
        out = self.apply_H0(vec)
        out[0] += vec[0] * self.c0 / self.epsilon ** self.ndim
        return out

    def apply_Hp(self, vec):
        return (vec.reshape([self.n1d] * self.ndim) * self.p2_over_2m).reshape(
            [self.n1d ** self.ndim]
        ) + self.c0 * np.sum(vec) / self.n1d ** self.ndim / self.epsilon ** self.ndim

    def apply_H0(self, vec):
        if self.cuda:
            return (
                cp.fft.ifftn(
                    cp.fft.fftn(cp.array(vec).reshape([self.n1d] * self.ndim))
                    * self.p2_over_2m_device
                )
                .reshape([self.n1d ** self.ndim])
                .get()
            )
        else:

            return np.fft.ifftn(
                np.fft.fftn(vec.reshape([self.n1d] * self.ndim)) * self.p2_over_2m
            ).reshape([self.n1d ** self.ndim])


h = FFT_H(20, 0.1, 3, -1.0, 0.5)


In [ ]:
solver = Solver(20, 0.1, 1.0, ndim_max=3, derivative_shifts=get_laplace_coefficients(4))

In [ ]:
energies = solver.get_energies(-1.0, 6)

In [ ]:
import scipy as sp

In [ ]:
neigs = 6
eigsp, vecsp = sp.sparse.linalg.eigsh(h.matp, int(neigs*1.2), which="SA")
eigs, vecs = sp.sparse.linalg.eigsh(h.mat, int(neigs*1.2), which="SA")

eigs.sort()
eigsp.sort()

eigsp, eigs = eigsp[:neigs], eigs[:neigs]

In [ ]:
eigs - eigsp

In [ ]:
eigs

In [ ]:
eigsp

In [ ]:
energies

In [ ]:
(energies - eigs)

In [ ]:
(energies / eigs)